In [32]:
import pandas as pd
from openai import OpenAI
from dataclasses import dataclass
from typing import List
import numpy as np
from collections import Counter
import re

In [48]:
@dataclass
class InformationStructure:
    information: str
    embedding: List[float]


information = [
    'Proběhla první onkologická konzultace.', 'Pacientka byla diagnostikována s karcinomem levého prsu.', 'Pacientka je po menopauze.', 'Nález se nachází v horním zevním kvadrantu levého prsu.', 'Na nálezu je kožní infiltrace.', 'V axile jsou hmatné uzliny.', 'Pacientka má dlouhodobé bolesti zad.', 'Pacientka má bolesti kloubů.', 'Pacientka má známou revmatoidní artritidu.', 'Pacientka používá francouzské hole při chůzi.', 'Bylo provedeno základní fyzikální vyšetření.', 'Proběhla kontrola dokumentace z mamologického pracoviště.', 'Závěr vyšetření uvádí suspektně lokálně pokročilý nádor prsu.', 'Je nutné dovyšetření.', 'Je nutný kompletní staging.',
    'Byla provedena klinicko-farmaceutická konzultace.', 'Byla zhodnocena chronická medikace včetně imunosupresiv.', 'Byla zhodnocena chronická medikace včetně analgetik.', 'Byla zhodnocena chronická medikace včetně suplementů.', 'Byla zjištěna snížená funkce jediné ledviny.', 'Byla zjištěna zvýšená hodnota kalcia.', 'Nebyly nalezeny zásadní interakce s plánovanou protinádorovou léčbou.', 'Bylo doporučeno sledování renálních funkcí.', 'Bylo doporučeno sledování elektrolytů.', 'Byla doporučena úprava dávek analgetik podle aktuálního stavu.',
    'Pacientka byla přijata na krátkodobý příjem na onkologickém oddělení kvůli asymptomatické hyperkalcémii.', 'Pacientka má metastatický karcinom prsu.', 'Pacientka má suspektní kostní postižení.', 'Byla provedena hydratace intravenózními infuzemi.', 'Podán byl redukovaný bolus bisfosfonátu vzhledem k monoledvině.', 'Kalcémie po terapii klesla.', 'Pacientka je klinicky stabilní.', 'Pacientka neměla dušnost.', 'Pacientka neměla febrilie.', 'Pacientka byla propuštěna domů.', 'Doporučen byl dostatečný příjem tekutin.', 'Doporučeno vysadit volně prodejné kalcium.',
    'Pacientka má HER2-pozitivní karcinom levého prsu.', 'Má diseminaci do skeletu.', 'Má diseminaci do jater.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Zvládá základní sebeobsluhu.', 'Pacientka subjektivně uvádí kolísající bolesti kostí.', 'Pacientka nemá nauzeu ani zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je pacientka kardiopulmonálně kompenzovaná.', 'Lokální rozsah tumoru v prsu je stacionární.', 'Axila obsahuje drobné tuhé uzliny.', 'Laboratorně je krevní obraz přiměřený.', 'Laboratorně jsou jaterní testy mírně zvýšené.', 'Pokračovat v aktuální léčbě.', 'Je nutné pečlivě monitorovat toxicitu léčby.', 'Je nutné pečlivě monitorovat bolest.',
    'Má HER2‑pozitivní karcinom levého prsu.', 'Má metastázy do skeletu.', 'Má metastázy do jater.', 'Pacientka je léčena paliativní systémovou léčbou.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Zvládá základní sebeobsluhu.', 'Subjektivně uvádí kolísající bolesti kostí.', 'Pacient nemá významnou nauzeu.', 'Pacient nemá zvracení.', 'Pacient nemá klidovou dušnost.', 'Fyzikálně je kardiopulmonálně kompenzovaný.', 'V prsu je lokální rozsah tumoru stacionární.', 'V axille jsou drobné tuhé uzliny.', 'Krevní obraz je laboratorně přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Závěr: pokračovat v aktuální léčbě.', 'Závěr: bude probíhat pečlivé monitorování toxicity léčby.', 'Závěr: bude probíhat pečlivé monitorování bolesti.',
    'Pacient je léčen cílenou anti-HER2 terapií.', 'Transtorakální echokardiografie byla provedena v rámci monitorace kardiální funkce.', 'Levá komora má normální velikost.', 'Globální systolická funkce levé komory je zachována.', 'Ejekční frakce byla odhadnuta na přibližně 60 %.', 'Regurgitace na aortální chlopni je nevýznamná.', 'Regurgitace na mitrální chlopni je nevýznamná.', 'Nebyly známky plicní hypertenze.', 'Perikard je bez patologického výpotku.', 'Kardiologicky není kontraindikace pokračovat v současném režimu léčby.', 'Doporučeno je pravidelné echokardiografické sledování.',
    'Případ byl projednán multidisciplinárním týmem bez přítomnosti pacientky.', 'Pacientka nebyla přítomna při jednání.', 'Má HER2 pozitivní karcinom levého prsu.', 'Postižení prsu je bifokální s kožní infiltrací.', 'Má mnohočetné kostní metastázy.', 'Má jaterní metastázy.', 'Není primárně indikována chirurgická léčba prsu.', 'Skelet je symptomaticky postižen.', 'Komise se shodla na paliativní systémové terapii.', 'Systémová terapie bude kombinací taxanu a dvojité blokády HER2.', 'K terapii bude doplněna kostně cílená léčba.',
    'Má HER2-pozitivní karcinom levého prsu s metastázemi do skeletu a do jater.', 'Pacientka podstupuje paliativní systémovou léčbu.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v oblasti páteře a pánevního skeletu.', 'Zvládá základní sebeobsluhu.', 'Pacientka subjektivně uvádí kolísající bolesti kostí.', 'Nemá významnou nauzeu ani zvracení.', 'Není přítomna klidová dušnost.', 'Fyzikálně je kardiopulmonálně kompenzovaná.', 'Lokální rozsah tumoru v prsu je stacionární.', 'Axilární uzliny jsou drobné a tuhé.', 'Laboratorně je krevní obraz přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Závěr: pokračovat v aktuální léčbě s pečlivým monitorováním toxicity a bolesti.',
    'Zhodnocena chronická medikace u pacienta s revmatoidní artritidou, včetně imunosupresiv.', 'Zhodnocena analgetika a suplementy.', 'Bylo upozorněno na sníženou funkci jediné ledviny.', 'Byla zjištěna zvýšená hladina kalcia.', 'Nebyly nalezeny zásadní interakce s plánovanou protinádorovou léčbou.', 'Bylo doporučeno sledovat renální funkce.', 'Bylo doporučeno sledovat elektrolyty.', 'Byla upravena dávka analgetik podle aktuálního stavu.',
    'Proběhlo komplexní onkologické vyšetření při kratší hospitalizaci.', 'Proběhla rekapitulace osobní anamnézy.', 'Proběhla rekapitulace rodinné anamnézy.', 'Proběhla rekapitulace gynekologické anamnézy.', 'Pacient měl dříve odstraněnou jednu ledvinu pro renální tumor.', 'Pacient má dlouhodobé revmatologické onemocnění.', 'Pacient má kostní metastázy karcinomu prsu.', 'Pacient má jaterní metastázy karcinomu prsu.', 'Pacient má osteoporózu.', 'Pacient má chronickou bolest zad.', 'Pokračování paliativní léčby bylo domluveno.', 'Byly domluveny pravidelné kontroly v onkologické ambulanci.', 'Proběhla krátká hospitalizace.',
    'Onkologická kontrola v průběhu paliativní systémové léčby pro HER2 pozitivní karcinom levého prsu s diseminací do skeletu a jater.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři a v oblasti pánevního skeletu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka udává kolísající bolesti kostí.', 'Pacientka nemá významnou nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je pacientka celkově kardiopulmonálně kompenzovaná.', 'Lokálně v prsu je stacionární rozsah tumoru.', 'Axila obsahuje drobné tuhé uzliny.', 'Krevní obraz je laboratorně přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Pacientka má pokračovat v aktuální léčbě za pečlivého monitorování toxicity a bolesti.',
    'Chirurgická ambulance provedla kontrolu po implantaci centrálního žilního portu.', 'Port byl implantován do oblasti horní hrudní stěny.', 'Lokalita portu je klidná.', 'Lokalita portu je bez zarudnutí.', 'Lokalita portu je bez sekrece.', 'Port je dobře palpačně přístupný.', 'Port je bez známek infekce.', 'Stehy byly odstraněny.', 'Pacientka byla poučena o péči o port.', 'Pacientka byla poučena o hlášení případných komplikací.', 'Hlášené případné komplikace zahrnují bolest.', 'Hlášené případné komplikace zahrnují zarudnutí.', 'Hlášené případné komplikace zahrnují febrilie.', 'Port byl ponechán k dlouhodobému používání pro systémovou léčbu.',
    'Diagnóza je HER2‑pozitivní karcinom levého prsu.', 'Diseminace do skeletu.', 'Diseminace do jater.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka uvádí kolísající bolesti kostí.', 'Pacientka nemá významnou nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je pacient kardiopulmonálně kompenzovaný.', 'Lokální rozsah tumoru v prsu je stacionární.', 'Axila obsahuje drobné tuhé uzliny.', 'Krevní obraz je laboratorně přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Pokračování v aktuální léčbě.', 'Bude prováděno pečlivé monitorování toxicity léčby.', 'Bude prováděno pečlivé monitorování bolesti.',
    'Pacientka má HER2‑pozitivní karcinom levého prsu.', 'Karcinom levého prsu má metastázy do skeletu.', 'Karcinom levého prsu se rozšířil do jater.', 'Pacientka je v paliativní systémové léčbě.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka subjektivně uvádí kolísající bolesti kostí.', 'Pacientka nemá významnou nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Pacientka je kardiopulmonálně kompenzovaná.', 'V prsu je stacionární rozsah tumoru.', 'V axile jsou drobné tuhé uzliny.', 'Krevní obraz je laboratorně přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Závěr: pokračovat v aktuální léčbě.', 'Závěr: monitorovat toxické účinky léčby.', 'Závěr: monitorovat bolest.',
    'Transtorakální echokardiografie byla provedena v rámci monitorace kardiální funkce při podané cílené anti-HER2 léčbě.', 'Levá komora má normální velikost.', 'Globální systolická funkce je zachována.', 'Ejekční frakce odhadovaná kolem 60 %.', 'Nevýznamná regurgitace na aortální chlopni.', 'Nevýznamná regurgitace na mitrální chlopni.', 'Není zjištěna plicní hypertenze.', 'Perikard je bez patologického výpotku.', 'Závěr: kardiologicky bez kontraindikace pokračování v současném režimu léčby.', 'Doporučeno pravidelné echokardiografické sledování.',
    'Má HER2 pozitivní karcinom levého prsu.', 'Karcinom levého prsu metastazuje do skeletu.', 'Karcinom levého prsu metastazuje do jater.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka subjektivně udává kolísající bolesti kostí.', 'Pacientka nemá nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Kardiopulmonální status je kompenzovaný.', 'Lokální rozsah tumoru v prsu je stacionární.', 'Axilární uzliny jsou drobně tuhé.', 'Krevní obraz je přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Pokračuje se v aktuální léčbě.', 'Je nutné pečlivé monitorování toxicity léčby.', 'Je nutné pečlivé monitorování bolesti.',
    'Spondylo-onkologická komise provedla konzultaci nálezu na páteři a pánevním skeletu.', 'Na zobrazovacím vyšetření byly popsány mnohočetná osteolytická ložiska.', 'Na zobrazovacím vyšetření byly popsány kompresivní fraktury obratlových těl.', 'Bylo identifikováno riziko patologické fraktury proximálního femuru.', 'Bylo doporučeno šetrné zatěžování.', 'Bylo doporučeno používání ortopedických pomůcek.', 'Bylo doporučeno zvážení stabilizačního výkonu.', 'Bylo doporučeno radioterapie na symptomatické úseky páteře.', 'Bylo doporučeno pokračování kostně cílené léčby.',
    'Proběhla telefonická konzultace upřesňující termíny kontrol a laboratorních odběrů.', 'Proběhlo probrání průběhu posledních dní se pacientkou nebo s rodinným příslušníkem.', 'Pacientka neměla horečku.', 'Nebyly zaznamenány nové neurologické příznaky.', 'Přetrvávají bolesti zad.', 'Účinek analgetik je dostatečný.', 'Doporučeno je nadále sledovat teplotu.', 'Doporučeno je kontaktovat pracoviště při náhlém zhoršení bolestí nebo při pádu.', 'Doporučeno je dostavit se k plánované kontrole včetně krevního obrazu a biochemie.',
    'Pacientka má HER2 pozitivní karcinom levého prsu.', 'Karcinom má diseminaci do skeletu.', 'Karcinom má diseminaci do jater.', 'Pacientka je na paliativní systémové léčbě.', 'Probíhá onkologická kontrola.', 'Pacientka přichází na vozíku.', 'Bolesti v páteři omezují mobilitu.', 'Bolesti v oblasti pánevního skeletu omezují mobilitu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka subjektivně uvádí kolísající bolesti kostí.', 'Pacientka nemá významnou nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je kardiopulmonálně kompenzovaná.', 'Rozsah tumoru v prsu je stacionární.', 'Axilární uzliny jsou drobně tuhé.', 'Laboratorní krevní obraz je přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Závěr je pokračovat v aktuální léčbě.', 'Bude prováděno pečlivé monitorování toxicity léčby.', 'Bolest bude pečlivě monitorována.',
    'Diagnóza je HER2 pozitivní karcinom levého prsu s diseminací do skeletu a jater.', 'Probíhá onkologická kontrola v průběhu paliativní systémové léčby.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři a v oblasti pánevního skeletu.', 'Zvládá základní sebeobsluhu.', 'Pacientka uvádí kolísavé bolesti kostí.', 'Pacientka nemá významnou nauzeu ani zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je kardiopulmonálně kompenzovaná.', 'V prsu je lokálně stacionární rozsah tumoru.', 'Axila obsahuje drobné tuhé uzliny.', 'Laboratorně je krevní obraz přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Závěr: pokračovat v aktuální léčbě za pečlivého monitorování toxicity a bolesti.',
    'Transtorakální echokardiografie byla provedena v rámci monitorace kardiální funkce při podané cílené anti-HER2 léčbě.', 'Levá komora má normální velikost.', 'Levá komora má zachovanou globální systolickou funkci.', 'Ejekční frakce byla odhadována kolem 60 procent.', 'Regurgitace na aortální chlopni je nevýznamná.', 'Regurgitace na mitrální chlopni je nevýznamná.', 'Nebyly zjištěny známky plicní hypertenze.', 'Perikard je bez patologického výpotku.', 'Závěr uvádí, že není kardiologicky kontraindikováno pokračovat v současném režimu léčby.', 'Doporučeno je pravidelné echokardiografické sledování.',
    'Pacientka má HER2 pozitivní karcinom levého prsu.', 'Karcinom levého prsu má diseminaci do skeletu.', 'Karcinom levého prsu má diseminaci do jater.', 'Pacientka je na onkologické kontrole.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka subjektivně udává kolísající bolesti kostí.', 'Pacientka nemá významnou nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je pacientka kardiopulmonálně kompenzovaná.', 'Lokálně v prsu je stacionární rozsah tumoru.', 'Axila obsahuje drobné tuhé uzliny.', 'Krevní obraz je laboratorně přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Bude pokračovat v aktuální léčbě.', 'Bude prováděno pečlivé monitorování toxicity a bolesti.',
    'Byla provedena transtorakální echokardiografie pro monitoraci kardiální funkce během cílené anti-HER2 léčby.', 'Levá komora má normální velikost.', 'Globální systolická funkce je zachována.', 'Ejekční frakce odhadovaná kolem 60 %.', 'Regurgitace na aortální chlopni je nevýznamná.', 'Regurgitace na mitrální chlopni je nevýznamná.', 'Nebyly zjištěny známky plicní hypertenze.', 'Perikard je bez patologického výpotku.', 'Závěr zní, že kardiologicky není kontraindikace pokračování v současném režimu léčby.', 'Doporučeno pravidelné echokardiografické sledování.',
    'Pacientka je léčená pro metastatický karcinom prsu.', 'Posouzen byl celkový interní stav.', 'Krevní tlak byl mírně zvýšený.', 'Pacientka je oběhově kompenzovaná.', 'Laboratorně jsou renální parametry mírně zhoršené, odpovídající chronickému stavu po nefrektomii.', 'Nebyly zjištěny překážky pro pokračování v protinádorové léčbě.', 'Byla doporučena úprava antihypertenzní terapie.', 'Byly doporučeny pravidelné kontroly u praktického lékaře.',
    'Pacientka má HER2 pozitivní karcinom levého prsu s diseminací do skeletu a do jater.', ' Pacientka je na paliativní systémové léčbě.', ' Pacientka je na onkologické kontrole.', ' Pacientka přijíždí na vozíku.', ' Mobilita je omezena bolestmi v páteři a v oblasti pánevního skeletu.', ' Zvládá základní sebeobsluhu.', ' Subjektivně udává kolísající bolesti kostí.', ' Nemá významnou nauzeu ani zvracení.', ' Není přítomna klidová dušnost.', ' Fyzicky je kardiopulmonálně kompenzovaná.', ' Lokálně v prsu je stacionární rozsah tumoru.', ' Axila obsahuje drobné tuhé uzliny.', ' Laboratorně je krevní obraz přiměřený.', ' Mírně zvýšené jaterní testy.', ' Závěr zní, že je třeba pokračovat v aktuální léčbě.', ' Léčba bude prováděna s pečlivým monitorováním toxicity a bolesti.',
    'Proběhla onko-hematologická konzultace po zobrazení mozku a dalších částí skeletu.', 'Byla potvrzena diseminace onemocnění s postižením mozkového parenchymu a mnohačetnými kostními metastázami.', 'Onko-hematolog doporučuje zachovat stávající systémovou léčbu.', 'Onko-hematolog doporučuje soustředit se na symptomatickou kontrolu bolesti a neurologických obtíží.', 'Případně doplnit lokální radioterapii na nejvíce symptomatická ložiska.',
    'Byla provedena klinicko-farmaceutická konzultace.', 'Zhodnocena byla chronická medikace včetně imunosupresiv při revmatoidní artritidě.', 'Zhodnocena byla chronická medikace obsahující analgetika.', 'Zhodnocena byla chronická medikace obsahující suplementy.', 'Bylo upozorněno na sníženou funkci jediné ledviny.', 'Bylo upozorněno na zvýšenou hodnotu kalcia.', 'Nebyla nalezena žádná zásadní interakce s plánovanou protinádorovou léčbou.', 'Bylo doporučeno sledování renálních funkcí.', 'Bylo doporučeno sledování elektrolytů.', 'Bylo doporučeno upravit dávky analgetik podle aktuálního stavu.',
    'Proběhla konzultace paliativního týmu.', 'Byla přítomna pacientka.', 'Byla přítomna blízká osoba.', 'Diskutovány byly možnosti domácí péče.', 'Probíhá úprava prostředí s ohledem na prevenci pádů.', 'Probíhá plánování budoucí léčby.', 'Bolesti kostí jsou hlavní obtíží.', 'Pacientka má únavu.', 'Pacientka má obavy z dalšího průběhu onemocnění.', 'Byla navržena optimalizace analgetické terapie.', 'Bylo zváženo zapojení domácí ošetřovatelské služby.', 'Byla poskytována podpora při zvládání psychické zátěže.',
    'Ambulantní kontrola probíhá po nedávné hospitalizaci a po úpravě analgetické léčby.', 'Pacientka popisuje mírné zlepšení zvládání běžných denních aktivit.', 'Přetrvává únava.', 'Přetrvává omezená chůze.', 'Nejsou žádné nové krvácivé projevy.', 'Stolice je bez patologie.', 'Močení je bez patologie.', 'Bylo dohodnuto pokračování v aktuálním analgetickém režimu.', 'Kontrola proběhne za několik týdnů.', 'V případě náhlého zhoršení stavu bude provedena kontrola dříve.',
    'Proběhla konzultace paliativního týmu.', 'Byla přítomna pacientka.', 'Byla přítomna blízká osoba.', 'Diskutovány byly možnosti domácí péče.', 'Diskutována byla úprava prostředí z hlediska prevence pádů.', 'Diskutováno bylo plánování budoucí léčby.', 'Hlavními obtížemi jsou bolesti kostí.', 'Hlavními obtížemi je únava.', 'Hlavními obtížemi jsou obavy z dalšího průběhu onemocnění.', 'Byla navržena optimalizace analgetické terapie.', 'Byla navržena možnost zapojení domácí ošetřovatelské služby.', 'Byla navržena podpora při zvládání psychické zátěže.',
    'Pacientce byly telefonicky sděleny výsledky posledních vyšetření.', 'Pacientce byl sdělen další postup.', 'Onemocnění je hodnoceno jako stabilizované.', 'Pacientce hrozí vysoké riziko komplikací při pádu.', 'Pacientce hrozí vysoké riziko komplikací při infekci.', 'Je kladen důraz na kvalitní analgezii.', 'Je kladen důraz na prevenci dekubitů.', 'Je poskytována podpora rodiny v péči o pacientku.', 'Byla nabídnuta možnost další konzultace dle potřeby.',
    'Radioterapeutické vyšetření je plánováno.', 'Důvodem léčby je paliativní ozáření bolestivých kostních ložisek v oblasti páteře a pánevního skeletu.', 'Případně bude provedena radioterapie prsu.', 'Pacientka má výrazné bolesti při chůzi.', 'Pacientka má bolesti při změně polohy.', 'Cílem léčby je úleva od bolesti.', 'Cílem léčby je snížení rizika patologických fraktur.', 'Je naplánováno krátké frakcionované schéma s ohledem na celkový stav a probíhající systémovou léčbu.', 'Ozařovaná oblast zahrnuje páteř a pánevní skelet.',
    'Proběhla telefonická konzultace.', 'Byly upřesněny termíny kontrol.', 'Byly upřesněny termíny laboratorních odběrů.', 'S pacientkou byl probrán průběh posledních dní.', 'Pacientka neměla horečku.', 'Pacientka neměla nové neurologické příznaky.', 'Přetrvávají bolesti zad.', 'Účinek analgetik je dostatečný.', 'Bylo doporučeno nadále sledovat teplotu.', 'Bylo doporučeno kontaktovat pracoviště při náhlém zhoršení bolestí.', 'Bylo doporučeno kontaktovat pracoviště při pádu.', 'Bylo doporučeno dostavit se k plánované kontrole včetně krevního obrazu a biochemie.',
    'Pacientka je po menopauze.', 'Pacientka má nově diagnostikovaný karcinom levého prsu.', 'Proběhla první onkologická konzultace.', 'Nález v horním zevním kvadrantu levého prsu s kožní infiltrací.', 'Hmatné uzliny v axile.', 'Subjektivně pacient uvádí dlouhodobé bolesti zad.', 'Subjektivně pacient uvádí bolesti kloubů při známé revmatoidní artritidě.', 'Pacientka chodí o francouzských holích.', 'Provedeno základní fyzikální vyšetření.', 'Provedena kontrola dokumentace z mamologického pracoviště.', 'Závěr vyšetření ukazuje suspektně lokálně pokročilý nádor prsu.', 'Je nutné dovyšetření.', 'Je nutné provést kompletní staging.',
    'Proběhla telefonická konzultace ohledně upřesnění termínů kontrol a laboratorních odběrů.', 'Byl probrán průběh posledních dní s pacientkou.', 'Do konverzace byl v případě potřeby zapojen rodinný příslušník.', 'Pacientka neměla horečku v posledních dnech.', 'Nebyly zaznamenány nové neurologické příznaky.', 'Přetrvávají bolesti zad.', 'Účinek analgetik je dostatečný.', 'Doporučeno nadále sledovat teplotu.', 'Doporučeno kontaktovat pracoviště při náhlém zhoršení bolestí.', 'Doporučeno kontaktovat pracoviště při pádu.', 'Doporučeno dostavit se k plánované kontrole.', 'Krevní obraz má být součástí plánované kontroly.', 'Biochemie má být součástí plánované kontroly.',
    'Transtorakální echokardiografie byla provedena pro monitoraci kardiální funkce při cílené anti-HER2 léčbě.', 'Levá komora má normální velikost.', 'Globální systolická funkce levé komory je zachována.', 'Ejekční frakce levé komory je odhadována na přibližně 60 %.', 'Regurgitace na aortální chlopni je nevýznamná.', 'Regurgitace na mitrální chlopni je nevýznamná.', 'Plicní hypertenze není přítomna.', 'Perikard je bez patologického výpotku.', 'Kardiologicky není kontraindikace pro pokračování v současném režimu léčby.', 'Bylo doporučeno pravidelné echokardiografické sledování.',
    'Proběhla telefonická konzultace s pacientkou (případně rodinným příslušníkem).', 'Byl probrán průběh posledních dní.', 'Nebyla horečka.', 'Neobjevily se nové neurologické příznaky.', 'Přetrvávají bolesti zad.', 'Účinek analgetik je dostatečný.', 'Doporučeno nadále sledovat teplotu.', 'Doporučeno kontaktovat pracoviště při náhlém zhoršení bolesti.', 'Doporučeno kontaktovat pracoviště při pádu.', 'Doporučeno dostavit se k plánované kontrole.', 'Doporučeno provést krevní obraz.', 'Doporučeno provést biochemii.',
    'Telefonická konzultace proběhla za účelem upřesnění termínů kontrol a laboratorních odběrů.', 'S pacientkou (případně rodinným příslušníkem) byl probrán průběh posledních dní.', 'Průběh posledních dní byl bez horeček.', 'Průběh posledních dní byl bez nových neurologických příznaků.', 'Přetrvávají bolesti zad.', 'Účinek analgetik je dostatečný.', 'Doporučeno nadále sledovat teplotu.', 'Doporučeno kontaktovat pracoviště při náhlém zhoršení bolestí nebo při pádu.', 'Doporučeno dostavit se k plánované kontrole včetně krevního obrazu a biochemie.',
    'Pacientka je nadále sledována v pravidelných intervalech.', 'Systémová léčba byla přizpůsobena aktuálnímu výkonostnímu stavu.', 'Přetrvává závislost na pomoci druhé osoby při některých denních činnostech.', 'Psychicky je pacientka stabilní.', 'Pacientka je informovaná o povaze onemocnění a o možnostech další péče.', 'Domluvena je další kontrola.', 'Je možné kontaktovat pracoviště při jakémkoli zhoršení.'
]

In [94]:
information = ['Důvod odeslání je posouzení možností radioterapie.',
 'Odesílající lékař pochází z dermatologické ambulance regionální nemocnice.',
 'Pracoviště odesílajícího týmu je regionální nemocnice.',
 'Pacientka má dlouhodobý defekt kůže na temeni hlavy po dřívějším úrazu a plastické operaci.',
 'V posledním roce kožní léze zvětšuje.',
 'Kožní léze krvácí.',
 'Z rány vytéká zapáchající sekret.',
 'Pacientka má časté bolesti hlavy.',
 'Pacientka má únavu.',
 'Pacientka má obtížný spánek.',
 'Karnofskyho index je 70 procent.',
 'Pacientka je při vědomí.',
 'Pacientka je orientovaná.',
 'Pacientka je spolupracující.',
 'Pacientka má pokrývku hlavy.',
 'Na kalvě parietálně vlevo přesahující střední čáru je nepravidelné ulcerované ložisko.',
 'Ložisko měří přibližně 14 × 14 cm.',
 'Ložisko má šedožlutý povlak.',
 'Ložisko obsahuje místy granulační tkáň.',
 'V centru defektu je patrná destrukce kosti kalvy.',
 'Okolní kůže je ztenčená.',
 'Okolní kůže je zarudlá.',
 'Regionální uzliny nejsou hmatné.',
 'Histologie z předchozí biopsie ukazuje invazivní dlaždicobuněčný karcinom kůže s ulcerací.',
 'Nádor je středně až málo diferencovaný.',
 'Nádor zasahuje do okrajů odebraného vzorku.',
 'CT kalvy ukazuje defekt kosti parietálně vlevo o šíři cca 5 cm.',
 'CT kalvy ukazuje známky zánětlivých změn v měkkých tkáních.',
 'CT kalvy ukazuje podezření na intrakraniální propagaci.',
 'RTG plic ukazuje bez známek ložiskových změn.',
 'Diagnostický souhrn uvádí pokročilý spinocelulárn í karcinom kalvy.',
 'Stav po starší traumatické a plastické operaci temene.',
 'Pacientka dosud byla sledována v domácí ošetřovatelské péči.',
 'Indikována je paliativní radioterapie v oblasti kalvy s cílem snížit bolest a sekreci.',
 'Ozařování bude provedeno na téměř celé kalvě včetně podložené mozkové tkáně pod defektem kosti.',
 'Frakcionace radioterapie je 5 × 2,0 Gy týdně.',
 'Celková dávka radioterapie je 50 Gy.',
 'Ošetřovatelské převazy budou v ambulanci radioterapie během celé léčby.',
 'Pacientka byla poučena o průběhu radioterapie a o akutních i pozdních nežádoucích účincích.',
 'Pacientka podepsala informovaný souhlas.',
 'Péče bude poskytnuta ambulantně ve spolupráci s domácí ošetřovatelskou službou.',
 'Závěr: zhoubný novotvar kůže NS [C44.9].',
 'Morfologie: dlaždicobuněčný karcinom.',
 'Morfologický kód: 8070/3.',
 'Péče bude poskytnuta ambulantně ve spolupráci s domácí ošetřovatelskou službou.',
 'Odeslány byly čtyři válečky tkáně o celkové délce 12 mm z rezistence v pravém prsu.',
 'Materiál byl uložen v 10% formalínu.',
 'Materiál byl kompletně zpracován.',
 'Ve všech vzorcích je přítomen invazivní karcinom s převážně solidní a kribriformní architekturou s výraznou extracelulární mukoprodukcí.',
 'Jaderný grade je střední.',
 'Mitózy jsou ojedinělé.',
 'V okolí ložisek nádorových buněk jsou nápadné proužky hyalinního vaziva.',
 'Myoepiteliální buňky nejsou přítomny.',
 'IHC p63 je negativní.',
 'Nález podporuje invazivní charakter léze.',
 'ER je pozitivní ve 100 % nádorových buněk se silnou jadernou reakcí.',
 'PR je cca 10 % nádorových buněk se slabou až střední jadernou reakcí.',
 'Ki67 proliferace je nízká, odhad 3 %. ',
 'Core biopsie pravého prsu ukazuje invazivní mucinózní karcinom, celulární typ, odpovídající středně diferencovanému nádoru (G2).',
 'Diferenciálně diagnosticky se zvažuje solidně papilární karcinom, pro stávající terapii bez významu.',
 'HER2 bude došetřen zvláštním vyšetřením.',
 'Topografie ICD-O-3 je C50.1.',
 'Morfologie ICD-O-3 je 8480/3.',
 'Příjem byl 4. 10. 2022.',
 'Provedeno bylo 5. 10. 2022.',
 'Vyšetřující patolog je specialista na mamární patologii.',
 'Parafínový blok byl z core biopsie pravého prsu.',
 'Imunohistochemická reakce byla provedena podle certifikovaného kitu s monoklonální protilátkou proti HER2.',
 'Hodnocena byla pouze membránová pozitivita v oblasti invazivního karcinomu.',
 'Membránová pozitivita v nádorových buňkách nebyla prokázána.',
 'IHC skóre bylo 0 (negativní).',
 'Interpretace uvedla, že karcinom prsu je HER2 negativní ve smyslu platných indikačních kritérií.',
 'FISH vyšetření není indikováno.',
 'Příjem proběhl dne 5. 10. 2022.',
 'Vyšetření provedeno dne 7. 10. 2022.',
 'Vyšetřující patolog je specialista na prediktivní markery.',
 'Jedná se o vyšetření HER2 u karcinomu prsu.',
 'Chirurgická mamární ambulance – první vyšetření pro nově diagnostikovaný nádor prsu.',
 'Pacientka je v průběhu paliativní radioterapie pro spinocelulární karcinom kalvy.',
 'Při hygieně zaznamenala hmatnou rezistenci v pravém prsu v oblasti za dvorcem.',
 'Praktický lékař ji odeslal do mamární poradny.',
 'Pacientka neudává bolesti v prsu.',
 'Pacientka nevšimla si sekrece z bradavky.',
 'Pacientka nemá celkové B-symptomy.',
 'Pacientka uvádí únavu z probíhající radioterapie a chronického defektu na hlavě.',
 'UZ prsu: oválná hypoechogenní léze v pravém prsu za dvorcem, cca 24 × 16 × 20 mm.',
 'UZ prsu: bez zjevné infiltrace pektorální fascie.',
 'Axilární uzliny normálního vzhledu.',
 'Core biopsie: invazivní mucinózní karcinom G2.',
 'Core biopsie: ER 100 %.',
 'Core biopsie: PR 10 %.',
 'Core biopsie: Ki67 3 %.',
 'Core biopsie: HER2 negativní (0).',
 'Klinické vyšetření: Prsy symetrické, ptotické.',
 'Vpravo v zóně za dvorcem v horním zevním kvadrantu hmatná tužší rezistence cca 3 cm.',
 'Rezistence poměrně dobře ohraničená.',
 'Rezistence volně posunlivá vůči hrudní stěně.',
 'Kůže bez známek retrakce.',
 "Kůže bez „peau d'orange“.",
 'V axilách nehmatné patologicky zvětšené uzliny.',
 'MG není zatím k dispozici.',
 'Bude doplněno oboustranné MG vyšetření.',
 'Závěr: Suspektní nádor pravého prsu – verifikovaný invazivní mucinózní karcinom.',
 'Závěr: klinicky cT2N0.',
 'Doplnit bilaterální mamografii.',
 'Doplnit standardní staging (UZ jater, RTG plic).',
 'V případě nepřítomnosti vzdálených metastáz plánujeme prs zachovávající výkon – parciální mastektomii pravého prsu s biopsií sentinelové uzliny (SNB).',
 'Definitivní rozhodnutí o rozsahu operace po doplnění zobrazení.',
 'Pacientka byla o plánovaném postupu informována.',
 'Pacientka souhlasí s plánovaným postupem.',
 'Pacientka žádá, pokud možno krátkou hospitalizaci.',
 'Pacientka žádá koordinaci termínu s ošetřovatelskou péčí o defekt na hlavě.',
 'Diagnóza: Zhoubný novotvar kůže NS [C44.9].',
 'Morfologie: Dlaždicobuněčný karcinom, kód 8070/3.',
 'Diagnóza doplňující: Spinocelulární karcinom kalvy.',
 'Stav po starším poranění kalvy a plastice.',
 'Pacient byl ošetřován v rámci domácí péče.',
 'Během radioterapie docházelo k převazům na pracoviště.',
 'Radioterapie kalvy byla provedena v rozsahu 25 × 2,0 Gy.',
 'Použita technika VMAT.',
 'Radioterapie ukončena dne 18. 10. 2022.',
 'Kožní toxicita byla max. G1.',
 'Během léčby byl nově diagnostikován invazivní mucinózní karcinom pravého prsu.',
 'ER byl 100 %.',
 'PR byl 10 %.',
 'HER2 byl 0.',
 'Ki67 nízké.',
 'Klinicky dle UZ rozměry ložiska přibližně 24 × 16 × 20 mm.',
 'Axily bez suspektních uzlin.',
 'MG dosud neprovedena.',
 'Průběh radioterapie: léčba byla dobře tolerována.',
 'Průběh radioterapie: došlo k subjektivní úlevě od bolesti v oblasti defektu.',
 'Průběh radioterapie: zápach a sekrece klesly jen částečně.',
 'V závěru léčby kůže v ozařovaném poli erytematózní s jemným olupováním.',
 'V závěru léčby: bez hlubokých nekróz.',
 'Karnofsky index při ukončení léčby byl 70 %.',
 'Subjektivní potíže: obavy z další léčby nádoru prsu.',
 'Jinak bez nových obtíží.',
 'Objektivně: kardiopulmonálně kompenzovaný.',
 'Objektivně: břicho klidné.',
 'Objektivně: dolní končetiny bez otoků.',
 'Na temeni hlavy přetrvává ulcerace s usurací kosti.',
 'Velikost ložiska se během RT významně nezměnila.',
 'Okraje ložiska jsou klidnější.',
 'Radioterapie kalvy ukončena dle plánu.',
 'Efekt léčby je parciální.',
 'Vzhledem k nově diagnostikovanému karcinomu prsu probíhá doplnění stagingu.',
 'Plánována prs zachovávající operace.',
 'Termín onkochirurgické kontroly v mamární ambulanci byl pacientce sdělen.',
 'Kontrola v radioterapeutické ambulanci za 4–6 týdnů s ohledem na stav defektu na hlavě.',
 'Mamografie byla provedena oboustranně.',
 'V levém prsu je středně denzní fibroglandulární struktura typu Tabár I.',
 'V levém prsu nebyly ložiskové změny.',
 'V pravém prsu je centrálně a do horního zevního kvadrantu neostře ohraničené ztluštění parenchymu odpovídající známému nádoru.',
 'V pravém prsu nebyly zjištěny podezřelé mikrokalcifikace.',
 'Axilární uzliny nejsou patrné.',
 'Ultrazvuk jater byl proveden.',
 'RTG plic byl proveden.',
 'Bez průkazu vzdálených metastáz.',
 'Známá malignita pravého prsu je invazivní mucinózní karcinom G2.',
 'Klinické stadium je cT2N0M0.',
 'Je stanoven plán prs zachovávajícího výkonu.',
 'Parciální mastektomie pravého prsu bude provedena po lokalizaci ložiska ultrazvukem.',
 'Biopsie sentinelové uzliny bude provedena.',
 'Pacientka preferuje hospitalizaci v pracovních dnech s možností zajištění doprovodu.',
 'Termín operace je předběžně stanoven na první polovinu listopadu.',
 'Anesteziologické vyšetření a přijetí proběhne den před výkonem.',
 'Vzhledem k souběžné léčbě rozsáhlého defektu kalvy je nutná úzká spolupráce s radioterapeutickou ambulancí.',
 'Je nutná také spolupráce s domácí péčí ohledně převazů.',
 'title":"InformationList","type":"object"}{',
 'title is InformationList, type object',
 'Diagnóza: invazivní mucinózní karcinom pravého prsu.',
 'Lokalizace nádoru: centrální oblast pravého prsu.',
 'Velikost nádoru dle zobrazovacích metod: 2–2,5 cm.',
 'Axilární uzliny: bez průkazu postižení axilárních uzlin.',
 'Vzdálené metastázy: bez známek vzdálených metastáz.',
 'ER receptor: 100 %.',
 'PR receptor: 10 %.',
 'HER2 receptor: 0.',
 'Ki67: nízké.',
 'Souběžná paliativní radioterapie: pro pokročilý spinocelulární karcinom kalvy.',
 'Komerbidita: významná komorbidita.',
 'Závěr: Indikována parciální mastektomie pravého prsu s biopsií sentinelové uzliny (SNB).',
 'Závěr: V případě peroperačního nálezu vícečetného či rozsáhlejšího ložiska je možné zvážit rozšíření výkonu, ale preferujeme zachování prsu.',
 'Závěr: Předpokládá se následná adjuvantní hormonoterapie.',
 'Závěr: Podle definitivního histologického výsledku též adjuvantní radioterapie prsu, pokud to dovolí celkový stav pacientky.',
 'Pacientka nebude komisi osobně přítomna.',
 'O závěrech bude informován ošetřující chirurg.',
 'Zapsal: člen mamární komise, onkochirurg.',
 'Předoperační anesteziologické vyšetření provedeno.',
 'Základní diagnóza: karcinom pravého prsu.',
 'Plánovaná parciální mastektomie s vyšetřením sentinelního uzlu (SNB).',
 'Dříve utrpěné těžké poranění hlavy s následnou plastickou operací.',
 'Nyní chronický defekt kůže.',
 'Proběhla paliativní radioterapie.',
 'Historie chronické pyelonefritidy.',
 'Pacientka je aktuálně bez obtíží z pyelonefritidy.',
 'Diagnóza cholecystolithiasis.',
 'Degenerativní postižení pohybového aparátu.',
 'Depresivní syndrom.',
 'Poruchy spánku.',
 'Dlouhodobé užívání trazodonu.',
 'Analgetika dle potřeby.',
 'Nesnášenlivost jednoho nesteroidního antiflogistika.',
 'Aktuálně bez projevů alergie na NSAID.',
 'Jiné známé lékové alergie nejsou.',
 'EKG vykazuje sinusový rytmus bez známek akutní ischemie.',
 'RTG plic bez výraznější patologie.',
 'Hemoglobin 120 g/l.',
 'Koagulační parametry v normě.',
 'Základní biochemie v mezích normy.',
 'Tělesná hmotnost 70 kg.',
 'Mírná hypertenze.',
 'Kardiopulmonálně kompenzovaná.',
 'Bez známek srdečního selhání.',
 'Bez klidové dušnosti.',
 'ASA II.',
 'Ze systémového hlediska je pacientka schopná podstoupit výkon v celkové anestezii.',
 'Rizika a možné komplikace operace i anestezie byly vysvětleny.',
 'Pacientka nemá dotazy.',
 'Souhlas s plánovaným výkonem byl pacientkou potvrzen podpisem.',
 'Premedikace večer před výkonem anxiolytikem.',
 'Ráno v den výkonu lehká premedikace per os.',
 'Ráno v den výkonu analgetikum per os.',
 'Jednalo se o vstupní konzultaci klinického farmaceuta.',
 'Byla ověřena aktuální chronická medikace podle dokumentace.',
 'Byl proveden rozhovor s pacientkou o její chronické medikaci.',
 'Polypragmazie nebyla zjištěna.',
 'Závažné interakce nebyly zjištěny.',
 'Nedostatečná adherence nebyla zjištěna.',
 'Pacientka má nízké riziko výskytu lékových problémů během plánované hospitalizace.',
 'Chronická medikace je ve vztahu k základním diagnózám vhodně nastavena.',
 'Vstupní medikace je akceptována z anesteziologického hlediska.',
 'Navržená premedikace z anesteziologického hlediska je akceptovatelná.',
 'Další sledování klinickým farmaceutem není rutinně nutné.',
 'Sledování je možné podle klinické potřeby.',
 'Zapsal: klinický farmaceut.',
 'V rodině se nádorová onemocnění prsu nevyskytují.',
 'Pacientka žije v domácnosti s rodinou.',
 'Pacientka je v důchodu.',
 'Pacientka dříve manuálně pracovala.',
 'Pacientka měla chronické onemocnění ledvin v minulosti.',
 'Pacientka prodělala úraz hlavy.',
 'Pacientka prodělala plastickou operaci.',
 'Pacientka má spinocelulární karcinom v oblasti jizvy po úrazu, léčba paliativní radioterapií.',
 'Pacientka má depresivní syndrom.',
 'Pacientka má poruchy spánku.',
 'Pacientka nemá akutní potíže.',
 'Pacientka nepociťuje klidovou bolest.',
 'Pacientka má lehkou úzkost před výkonem.',
 'Močení je pravidelné.',
 'Stolice je pravidelná.',
 'Pacientka nemá dušnost.',
 'Pacientka nemá stenokardii.',
 'Výška pacientské je 173 cm.',
 'Hmotnost pacientské je 70 kg.',
 'Tlak 150/85 mmHg.',
 'Pulz 80/min.',
 'Pacientka je lucidní.',
 'Pacientka je orientovaná.',
 'Dýchání je klidné.',
 'Srdeční akce je pravidelná.',
 'Břicho je klidné.',
 'Dolní končetiny bez otoků.',
 'Prsy jsou volumózní a ptotické.',
 'V pravém prsu za dvorcem je hmatná rezistence cca 3 cm, pohyblivá, bez kožních změn.',
 'Axily bez hmatných patologických uzlin.',
 'Na temeni hlavy je rozsáhlý defekt po spinocelulárním karcinomu.',
 'Defekt na hlavě je pravidelně převazován.',
 'Diagnóza: karcinom pravého prsu, klinicky cT2N0M0.',
 'Provedena parciální mastektomie pravého prsu a biopsie sentinelové uzliny.',
 'Operační výkon proběhl bez komplikací.',
 'Pooperační průběh byl bez komplikací.',
 'Pacientka je afebrilní.',
 'Pacientka je hemodynamicky stabilní.',
 'Drenáže byly postupně odstraněny.',
 'Rány jsou klidné.',
 'Horní končetina bez otoku a bez poruch hybnosti.',
 'Defekt na temeni hlavy byl převazován ve spolupráci s ošetřovatelským personálem.',
 'Propuštěna do domácí péče ve stabilizovaném stavu.',
 'Rány v oblasti prsu možno sprchovat.',
 'Rány v oblasti prsu ponechat volně.',
 'Vhodné nošení podprsenky.',
 'Kontrola u praktického lékaře do 3 dnů od propuštění.',
 'Další pracovní neschopnost dle stavu.',
 'Kontrola v mamární chirurgické ambulanci; výsledek definitivní histologie bude sdělen samostatně.',
 'Dočasně bude podávána profylaxe nízkomolekulárním heparinem.',
 'Analgetika dle potřeby.',
 'Převazy defektu kalvy nadále provádět domácí péčí.',
 'Zajištěny potřebné pomůcky.',
 'Pacientka je odeslána na kontrolu stavu po radioterapii kalvy.',
 'Pacientka je odeslána na kontrolu stavu po parciální mastektomii pravého prsu.',
 'Důvodem odeslání je zhoršená soběstačnost.',
 'Pacientka uvedla pád v domácím prostředí s poraněním levého zápěstí.',
 'Poranění levého zápěstí je nyní v dlaze.',
 'Po pádu je pacientka výrazně méně soběstačná.',
 'Většinu dne tráví na lůžku.',
 'Potřebuje pomoc rodiny.',
 'Rána na prsu ji nebolí.',
 'Obtěžuje ji spíše přetrvávající defekt na hlavě.',
 'Karnofského index činí 40–50 procent.',
 'Kardiopulmonálně je stav kompenzovaný.',
 'Pacientka byla převezena transportní službou na sedačce.',
 'Na levé horní končetině je dlahová fixace zápěstí.',
 'Na kalvě přetrvává rozsáhlý defekt s ulcerací a sekrecí.',
 'Velikost defektu na kalvě není jednoznačně regresní.',
 'V oblasti pravého prsu je pooperační jizva v centrální části.',
 'Jizva v oblasti pravého prsu je klidná.',
 'V oblasti pravého prsu není zarudnutí.',
 'Nebyly zjištěny známky recidivy v oblasti pravého prsu.',
 'Axily jsou bez hmatného nálezu.',
 'Pokročilý spinocelulární karcinom kalvy je stav po paliativní radioterapii.',
 'Efekt paliativní radioterapie kalvy je částečný.',
 'Ulcerace na kalvě nadále přetrvává.',
 'Nádorová duplicita je invazivní mucinózní karcinom pravého prsu.',
 'Pacientka je po parciální mastektomii a SNB.',
 'Definitivní histologický nález dosud není k dispozici.',
 'Další onkologický plán bude určen na základě pooperační histologie a doporučení mamární komise.',
 'Kvůli výraznému snížení soběstačnosti je indikována každodenní ošetřovatelská péče blízkou osobou.',
 'Vzhledem k omezené mobilitě a schopnosti komunikace je nadále indikovaný převoz sanitním vozem s doprovodem.',
 'Prioritou je symptomatická a ošetřovatelská péče o defekt na hlavě.',
 'Zajištění bezpečnosti v domácím prostředí je prioritou.',
 'Další radioterapeutické postupy budou zvažovány podle celkového stavu a závěrů mamární komise.',
 'Chirurgická kontrola proběhla 14 dní po parciální mastektomii se SNB pravého prsu.',
 'Pacientka nemá potíže s prsem.',
 'Není známka infekce u prsu.',
 'Není krvácení u prsu.',
 'V mezidobí došlo k pádu s frakturou levého předloktí.',
 'Fraktura levého předloktí byla ošetřena v regionální nemocnici.',
 'Má nyní sádrovou fixaci.',
 'Pooperační rány na prsu i v axile jsou klidné.',
 'Nebylo z ran zaznamenáno sekrece.',
 'Není hematom na ranách.',
 'Kožní stehy byly extrahovány.',
 'Otok horní končetiny nebyl pozorován.',
 'Hybnost horní končetiny je zatím mírně omezena kvůli bolesti.',
 'Definitivní histologické vyšetření resekátu prsu a sentinelové uzliny dosud není k dispozici.',
 'Hojení po chirurgické stránce je příznivé.',
 'Je možný návrat k běžnému dennímu režimu s omezením větší fyzické zátěže.',
 'Je možné zohlednit frakturu levé horní končetiny.',
 'Další onkologický postup bude stanoven na základě výsledku histologie v rámci mamární komise.',
 'Pacientka bude telefonicky informována o doporučeném postupu a termínu kontroly.',
 'Z důvodu omezené mobility bude zajištěn transport.',
 'Kontrola v chirurgické ambulanci proběhne pouze v případě komplikací.',
 'Komplikace mohou zahrnovat horečku, zarudnutí rány a sekreci.',
 'Příjem: 11. 11. 2022.',
 'Provedeno: 2. 12. 2022.',
 'Vyšetřující patolog: mamární specialista.',
 'Operační výkon: parciální mastektomie pravého prsu.',
 'Makroskopický nález: orientovaný resekát pravého prsu 60 × 60 × 40 mm s přiloženým označením okrajů.',
 'Makroskopický nález: centrální tužší ložisko 23 × 18 mm promítající se k mediálním a kraniálním okrajům.',
 'Makroskopický nález: vzdálenost tumoru od všech resekčních okrajů minimálně 10 mm.',
 'Makroskopický nález: přilehlý úsek fascie bez makroskopické infiltrace.',
 'Sentinelová uzlina: jedna uzlina velikosti cca 10 mm.',
 'Mikroskopický nález: invazivní karcinom s výraznou extracelulární mukoprodukcí, solidně kribriformní stavby.',
 'Nádorové buňky: tvoří okrouhlé hnízda oddělená proužky vaziva.',
 'Jádra: mírně variabilní; mitózy do 3/10 HPF.',
 'Angioinvaze: neprokázána.',
 'Laterálně: 0,5 mm od resekční linie drobný fokus duktálního karcinomu in situ (DCIS) kribriformní stavby G1 o velikosti asi 2 mm.',
 'Nález: hraniční s atypickou duktální hyperplazií.',
 'Resekční linie: negativní pro invazivní komponentu.',
 'Sentinelová uzlina: jedna mízní uzlina bez metastatické infiltrace.',
 'Imunohistochemie: Prediktivní faktory ER, PR, HER2 vyšetřeny z core biopsy: ER 100 %, PR 10 %, HER2 0.',
 'Ki67: 7 %.',
 'Závěr: invazivní mucinózní karcinom prsu, celulární typ, G2, s mikrofokusem DCIS G1 v laterálním okraji (0,5 mm od resekční linie).',
 'Resekční linie pro invazivní karcinom negativní (R0).',
 'Angioinvaze: neprokázána.',
 'Sentinelová uzlina: bez metastatického postižení (0/1).',
 'KLASIFIKACE: Topografie (ICD-O-3) C50.4 (pravý prs, horní zevní kvadrant).',
 'Morfologie (ICD-O-3): 8480/3.',
 'pTNM (8. vydání): pT2 (23 mm) pN0(i−)(sn) M0, L0, V0, R0.',
 'Zápis se týká pooperačního období mamární komise.',
 'K dispozici jsou definitive histologický protokol po parciální mastektomii a SNB.',
 'K dispozici je předchozí dokumentace.',
 'Invazivní mucinózní karcinom pravého prsu byl klasifikován jako G2.',
 'Tumor má velikost 23 mm.',
 'ER je pozitivní na 100 %.',
 'PR je pozitivní na 10 %.',
 'HER2 je negativní.',
 'Ki67 proliferativní index je 7 %.',
 'Byla provedena R0 resekce invazivní komponenty.',
 'V těsné blízkosti laterálního okraje byl nalezen mikrofokus DCIS G1.',
 'Sentinelová uzlina nebyla metastatická (0/1).',
 'Klinický stav pacientky je ovlivněn pokročilou malignitou kalvy.',
 'Pacientka byla po paliativní radioterapii.',
 'Pacientka má sníženou soběstačnost.',
 'Doporučena adjuvantní hormonoterapie.',
 'Zahájit tamoxifen adjuvantně.',
 'Po zahájení má následovat přechod na inhibitor aromatázy podle tolerance a vývoje.',
 'Celková předpokládaná délka adjuvantní hormonoterapie je 7–8 let.',
 'Konečná indikace radioterapie bude rozhodnuta radioterapeutem při osobní konzultaci.',
 'Standardně by byla indikována adjuvantní radioterapie pravého prsu po prs zachovávajícím výkonu.',
 'U této pacientky je nutno zohlednit celkový stav.',
 'Chemoterapie vzhledem k nízkému proliferačnímu indexu není doporučena.',
 'Chemoterapie vzhledem k komorbiditám není doporučena.',
 'Pacientka nebyla jednání přítomna.',
 'O závěrech bude informována ošetřujícím onkologem.',
 'Zapsal: člen mamární komise.',
 'Důvod odeslání je posouzení indikace adjuvantní radioterapie pravého prsu po parciální mastektomii.',
 'Pacientka má nádorovou duplicitu.',
 'Pacientka má omezenou soběstačnost.',
 'Pacientka udává únavu.',
 'Pacientka má obtížnější přesuny v domácnosti.',
 'Pacientka tráví většinu času na lůžku.',
 'Hlava bolí při převazech.',
 'Občasné bolesti tlumí analgetiky.',
 'V oblasti prsu nejsou výrazné obtíže.',
 'Karnofskyho index činí 60–70 %.',
 'Pacientka je oběhově kompenzovaná.',
 'Není klidová dušnost.',
 'Pacientka byla převezena na sedačce.',
 'Po operační nález na pravém prsu je klidný.',
 'Nález bez známek recidivy.',
 'Jizvy jsou dobře zhojené.',
 'Na kalvě přetrvává ulcerovaný defekt.',
 'Defekt má rozsah zhruba stacionární.',
 'Defekt je spojen s chronickou sekrecí.',
 'Spinocelulární karcinom kalvy byl diagnostikován.',
 'Stav po paliativní radioterapii 25×2,0 Gy.',
 'Ulcerace kalvy přetrvává.',
 'Pacient podstupuje symptomatickou léčbu.',
 'Invazivní mucinózní karcinom pravého prsu.',
 'Stav po parciální mastektomii a SNB.',
 'pT2N0M0.',
 'ER 100 %.',
 'PR 10 %.',
 'HER2 0.',
 'Ki67 7 %.',
 'R0 resekce.',
 'Mikrofokus DCIS.',
 'S ohledem na duplicitní nádorové onemocnění je adjuvantní ozáření prsu v tomto stadiu nedoporučováno.',
 'Očekávaný přínos adjuvantní radioterapie prsu je nízký.',
 'Zátěž radioterapie prsu je vysoká vzhledem k defektu na kalvě a snížené soběstačnosti.',
 'Upřednostňujeme zahájení hormonální léčby podle interního stavu.',
 'Budeme provádět pečlivou klinickou kontrolu prsu.',
 'Pacientce bude doporučeno zahájení hormonoterapie po stabilizaci celkového stavu ve spolupráci s klinickým onkologem.',
 'Nadále trvá potřeba pravidelných převazů defektu kalvy.',
 'Probíhá symptomatická léčba bolesti.',
 'Dispenzární kontrola probíhá na radioterapeutické ambulanci.',
 'Karnofskyho index je přibližně 50 %.',
 'Pacientka nemá výrazné klidové bolesti.',
 'Bolest se zhoršuje při převazu defektu na hlavě.',
 'Bolest se zhoršuje při delším ležení na postižené straně.',
 'Pacientka udává točení hlavy.',
 'Z důvodu kognitivního deficitu není schopna plně předávat informace.',
 'Doprovod doplňuje anamnestické údaje (rodinný příslušník).',
 'Hormonoterapie pro karcinom prsu dosud nezahájena.',
 'Vzhledem k celkovému stavu byla léčba opakovaně odložena.',
 'Pacientka je na transportní sedačce.',
 'Kardiopulmonálně je kompenzovaná.',
 'Na temeni hlavy přetrvává defekt s usurací kosti.',
 'Rozsah defektu spíše progreduje.',
 'Ulcerace zasahuje většinu kalvy.',
 'Kůže v okolí je zarudlá.',
 'Kůže v okolí je místy macerovaná.',
 'Jizvy po operaci pravého prsu jsou klidné.',
 'V oblasti pravého prsu není hmatná recidiva.',
 'V oblasti pravého prsu nebyla zjištěna nová ložiska.',
 'Axilární oblasti jsou bez suspektních uzlin.',
 'Pokročilý spinocelulární karcinom kalvy po paliativní radioterapii.',
 'Nyní dominují ulcerace a lokální obtíže u spinocelulárního karcinomu kalvy.',
 'Pravé prso má stav po parciální mastektomii a SNB.',
 'Pravé prso má nádor pT2N0M0.',
 'Nádor pravého prsu je hormonálně senzitivní.',
 'Adjuvantní radioterapie nebyla indikována.',
 'Hormonální léčba dosud nezahjena vzhledem k celkovému stavu.',
 'Nadále se provádějí převazy defektu na hlavě cestou domácí ošetřovatelské péče.',
 'Doporučena změna krytí s využitím moderních antiseptických a vlhkých hojení podporujících materiálů.',
 'Důsledná analgezie – transdermální opioidy v kombinaci s krátkodobě působícím analgetikem před převazy.',
 'Adjuvantní hormonoterapie vzhledem k omezené prognóze a zátěži léčby není indikována.',
 'Dominantní je symptomatická a paliativní péče.',
 'Další sledování probíhá v péči praktického lékaře a regionální onkologie; naše pracoviště je k dispozici ke konzultaci.',
 'Chronická kožní toxicita po RT je G1–G2 v oblasti kalvy.']

In [95]:
client = OpenAI()

response = client.embeddings.create(
    input=information,
    model="text-embedding-3-large"
)

In [96]:
information_nodes = []

for idx, res in enumerate(response.data):
    information_nodes.append(InformationStructure(information[idx], res.embedding))

In [97]:
def semantic_search(
    search_term: str,
    information_nodes: List[InformationStructure],
    client: OpenAI,
    threshold: float = 0.5,
    semantic_weight: float = 0.75,
    syntactic_weight: float = 0.25
) -> List[InformationStructure]:
    """
    Perform hybrid semantic search combining semantic similarity (embeddings) 
    and syntactic similarity (BM25-like).
    
    Args:
        search_term: The search query
        information_nodes: List of InformationStructure objects to search through
        client: OpenAI client for generating embeddings
        threshold: Minimum combined score to include a result (default: 0.5)
        semantic_weight: Weight for semantic similarity (default: 0.75)
        syntactic_weight: Weight for syntactic similarity (default: 0.25)
    
    Returns:
        List of InformationStructure objects that match above the threshold,
        sorted by combined score (descending)
    """
    # Get embedding for search term
    search_response = client.embeddings.create(
        input=[search_term],
        model="text-embedding-3-large"
    )
    search_embedding = np.array(search_response.data[0].embedding)
    
    # Normalize search embedding
    search_embedding_norm = search_embedding / np.linalg.norm(search_embedding)
    
    # Prepare documents for BM25
    documents = [node.information for node in information_nodes]
    
    # Tokenize documents and query
    def tokenize(text: str) -> List[str]:
        # Simple tokenization: lowercase and split on non-word characters
        tokens = re.findall(r'\b\w+\b', text.lower())
        return tokens
    
    tokenized_docs = [tokenize(doc) for doc in documents]
    tokenized_query = tokenize(search_term)
    
    # Calculate document frequencies for BM25
    doc_freqs = Counter()
    for doc_tokens in tokenized_docs:
        doc_freqs.update(set(doc_tokens))
    
    num_docs = len(documents)
    avg_doc_length = np.mean([len(tokens) for tokens in tokenized_docs])
    
    # BM25 parameters
    k1 = 1.5
    b = 0.75
    
    def bm25_score(query_tokens: List[str], doc_tokens: List[str]) -> float:
        """Calculate BM25 score for a document given query tokens."""
        score = 0.0
        doc_length = len(doc_tokens)
        doc_token_counts = Counter(doc_tokens)
        
        for term in query_tokens:
            if term in doc_token_counts:
                # Term frequency in document
                tf = doc_token_counts[term]
                # Inverse document frequency
                idf = np.log((num_docs - doc_freqs.get(term, 0) + 0.5) / (doc_freqs.get(term, 0) + 0.5))
                # BM25 formula
                numerator = tf * (k1 + 1)
                denominator = tf + k1 * (1 - b + b * (doc_length / avg_doc_length))
                score += idf * (numerator / denominator)
        
        return score
    
    # Calculate scores for each document
    results = []
    for idx, node in enumerate(information_nodes):
        # Semantic similarity (cosine similarity)
        doc_embedding = np.array(node.embedding)
        doc_embedding_norm = doc_embedding / np.linalg.norm(doc_embedding)
        semantic_score = np.dot(search_embedding_norm, doc_embedding_norm)
        
        # Normalize semantic score to [0, 1] (cosine similarity is already in [-1, 1])
        semantic_score_normalized = (semantic_score + 1) / 2
        
        # Syntactic similarity (BM25) - will normalize after calculating all scores
        syntactic_score = bm25_score(tokenized_query, tokenized_docs[idx])
        
        results.append((node, semantic_score_normalized, syntactic_score))
    
    # Normalize BM25 scores
    if results:
        max_bm25 = max([score for _, _, score in results]) or 1
        min_bm25 = min([score for _, _, score in results]) or 0
        
        # Normalize BM25 scores to [0, 1]
        normalized_results = []
        for node, sem_score, syn_score in results:
            if max_bm25 > min_bm25:
                syntactic_score_normalized = (syn_score - min_bm25) / (max_bm25 - min_bm25)
            else:
                syntactic_score_normalized = 0.0
            
            # Combined score
            combined_score = (semantic_weight * sem_score + 
                        syntactic_weight * syntactic_score_normalized)
            
            if combined_score >= threshold:
                normalized_results.append((node, combined_score, sem_score, syntactic_score_normalized))
        
        results = normalized_results
    else:
        results = []
    
    # Sort by combined score (descending)
    results.sort(key=lambda x: x[1], reverse=True)
    
    # Return only the InformationStructure objects
    return [result[0] for result in results]


In [116]:
# Example usage
results = semantic_search(
    search_term="TNM klasifikace",
    information_nodes=information_nodes,
    client=client,
    threshold=0.8,
    semantic_weight=0.75,
    syntactic_weight=0.25
)

# Display results
for result in results:
    print(f"- {result.information}")


- KLASIFIKACE: Topografie (ICD-O-3) C50.4 (pravý prs, horní zevní kvadrant).


In [101]:
info = [result.information.strip() for result in results]

In [92]:
set(info)

{'Byla potvrzena diseminace onemocnění s postižením mozkového parenchymu a mnohačetnými kostními metastázami.',
 'Diagnóza je HER2 pozitivní karcinom levého prsu s diseminací do skeletu a jater.',
 'Diagnóza je HER2‑pozitivní karcinom levého prsu.',
 'Diseminace do jater.',
 'Diseminace do skeletu.',
 'Důvodem léčby je paliativní ozáření bolestivých kostních ložisek v oblasti páteře a pánevního skeletu.',
 'Karcinom levého prsu metastazuje do jater.',
 'Karcinom levého prsu metastazuje do skeletu.',
 'Karcinom levého prsu má diseminaci do jater.',
 'Karcinom levého prsu má diseminaci do skeletu.',
 'Karcinom levého prsu má metastázy do skeletu.',
 'Karcinom levého prsu se rozšířil do jater.',
 'Karcinom má diseminaci do jater.',
 'Karcinom má diseminaci do skeletu.',
 'Lokální rozsah tumoru v prsu je stacionární.',
 'Lokálně v prsu je stacionární rozsah tumoru.',
 'Má HER2 pozitivní karcinom levého prsu.',
 'Má HER2-pozitivní karcinom levého prsu s metastázemi do skeletu a do jater.',


In [5]:
import requests
import json

with open('tmp.json', 'r', encoding='utf-8') as f:
    bundle = json.load(f)

response = requests.post(
    "http://localhost:32783/csp/healthshare/demo/fhir/r4",
    json=bundle,  # This handles encoding properly
    headers={
        "Content-Type": "application/fhir+json; charset=utf-8",
        "Accept": "application/fhir+json"
    },
    auth=("_SYSTEM", "ISCDEMO")
)

In [6]:
response

<Response [200]>